# задание 2

## описание задачи

В этом задании необходимо достичь максимального качества предсказания, используя навыки полученные за 4 недели обучения

Кто достигнет максимального значения на тестовой выборке, получит *15 баллов*

Пожалуйста, оформляйте ноутбук аккуратно. Все выводы подписывайте, оформляйте заголовки.

Для справедливой оценки все должны использовать одну и ту же часть исходного датасета в качестве тестового набора данных. Используйте разбиение приведенное ниже


```
train_test_split(X, y, test_size = 0.3, random_state = 69)
```
И модель, с которой вы работаете - это линейная регрессия. Другие алгоритмы не используйте.

Метрика, которую вы должны максимизировать

```
r2_score(y_test, y_pred) и root_mean_squared_error(y_test, y_pred)
```

Целевая переменная - *count*

Обратите внимание на столбцы - 'casual', 'registered'. Эти столбцы в сумме дают целевую переменную, по этой причине их надо удалить. Они линейно зависимы.

Используйтие средства библиотеки sklearn, внимательно изучите ее. Существует огромное количество методов для выполнения каждого из этапов, не бойтесь использовать методы неприведенные в лекции

Этапы, которые необходимо проделать в работе для успешного достижения результата:
* EDA (Исследовать данные, понять с чем имеете дело, наработать идеи для генерации фичей и их обработки, обязательно корреляционные матрицы и графики с hue=классы объектов)
* Анализ выбросов и их обработка (в задачах классификации выброс - это объект с таким признаковым описанием, которое отличается очень сильно от типичного и больше соответствует другому классу)
* Генерация новых фичей
* Обработка пропусков (вы можете сгенерировать новые фичи с пропусками, тогда надо подумать об их обработке)
* Обработка категориальных признаков
* Масштабирование вещественных признаков
* Трансформирование таргета
* Нелинейные автоматические трансформации признаков (Kernel Trick, Transfomers)
* Подбор параметров модели (можете менять не только константы, но и оптимизационные алгоритмы и методы регуляризации)
* Отбор признаков
* Обязательно: Составление пайплайна обучения
* Обязательно: Постройте график зависимости y_pred от y_true в разбиении на test и train. Предсказание идеального алгоритма даст прямую y=x, посмотрите какой график получается у вас.

## Bike Sharing Demand

По историческим данным о прокате велосипедов и погодным условиям необходимо оценить спрос на прокат велосипедов.

В наборе признаков присутсвуют вещественные, категориальные, и бинарные данные.

In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [2]:
X = pd.read_csv("bike_sharing_demand.csv")
y = X["count"]

X.drop(
    columns=["casual", "registered", "count"],
    inplace=True,
)

X.head(5)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0


In [3]:
# Предполагаем, что у нас есть столбец 'datetime' в формате datetime
def extract_datetime_features(df, datetime_column):
    # Создаем копию DataFrame
    df = df.copy()

    # Извлекаем различные временные характеристики
    df["hour"] = df[datetime_column].dt.hour
    df["day"] = df[datetime_column].dt.day
    df["month"] = df[datetime_column].dt.month
    df["year"] = df[datetime_column].dt.year
    df["dayofweek"] = df[datetime_column].dt.dayofweek

    # Можно добавить циклические признаки для часов и месяцев
    df["hour_sin"] = np.sin(2 * np.pi * df["hour"] / 24)
    df["hour_cos"] = np.cos(2 * np.pi * df["hour"] / 24)
    df["month_sin"] = np.sin(2 * np.pi * df["month"] / 12)
    df["month_cos"] = np.cos(2 * np.pi * df["month"] / 12)

    # Удаляем исходный столбец datetime
    df.drop(columns=[datetime_column], inplace=True)

    return df


X["datetime"] = pd.to_datetime(X["datetime"])

# Применяем преобразование
X_dt_done = extract_datetime_features(X, "datetime")

X_dt_done

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,hour,day,month,year,dayofweek,hour_sin,hour_cos,month_sin,month_cos
0,1,0,0,1,9.84,14.395,81,0.0000,0,1,1,2011,5,0.000000,1.000000,5.000000e-01,0.866025
1,1,0,0,1,9.02,13.635,80,0.0000,1,1,1,2011,5,0.258819,0.965926,5.000000e-01,0.866025
2,1,0,0,1,9.02,13.635,80,0.0000,2,1,1,2011,5,0.500000,0.866025,5.000000e-01,0.866025
3,1,0,0,1,9.84,14.395,75,0.0000,3,1,1,2011,5,0.707107,0.707107,5.000000e-01,0.866025
4,1,0,0,1,9.84,14.395,75,0.0000,4,1,1,2011,5,0.866025,0.500000,5.000000e-01,0.866025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10881,4,0,1,1,15.58,19.695,50,26.0027,19,19,12,2012,2,-0.965926,0.258819,-2.449294e-16,1.000000
10882,4,0,1,1,14.76,17.425,57,15.0013,20,19,12,2012,2,-0.866025,0.500000,-2.449294e-16,1.000000
10883,4,0,1,1,13.94,15.910,61,15.0013,21,19,12,2012,2,-0.707107,0.707107,-2.449294e-16,1.000000
10884,4,0,1,1,13.94,17.425,61,6.0032,22,19,12,2012,2,-0.500000,0.866025,-2.449294e-16,1.000000


values before and after are equal, so there's no missing values

In [4]:
print("before dropping missing values: ", X_dt_done.shape)
print("after: ", X_dt_done.dropna().shape)

before dropping missing values:  (10886, 17)
after:  (10886, 17)


In [5]:
numeric_cols = [
    "temp",
    "atemp",
    "humidity",
    "windspeed",
    "hour_sin",
    "hour_cos",
    "month_sin",
    "month_cos",
]
categorical_cols = list(set(X.columns.values.tolist()) - set(numeric_cols))

print("numeric cols: ", numeric_cols)
print("categorial cols: ", categorical_cols)

X_dt_done = X_dt_done[numeric_cols]
X_cat = X[categorical_cols]

numeric cols:  ['temp', 'atemp', 'humidity', 'windspeed', 'hour_sin', 'hour_cos', 'month_sin', 'month_cos']
categorial cols:  ['workingday', 'weather', 'holiday', 'season', 'datetime']


In [6]:
enc = OneHotEncoder(handle_unknown="ignore", drop="first", sparse_output=False)
X_cat = enc.fit_transform(X_cat)

In [7]:
(X_dt_done_train, X_dt_done_test, y_train, y_test) = train_test_split(
    X_dt_done, y, test_size=0.3, random_state=69
)

(X_cat_train, X_cat_test) = train_test_split(X_dt_done, test_size=0.3, random_state=69)

X_train = np.concatenate([X_dt_done_train, X_cat_train], axis=1)
X_test = np.concatenate([X_dt_done_test, X_cat_test], axis=1)

X_train, X_test

(array([[ 2.46000000e+01,  3.10600000e+01,  4.30000000e+01, ...,
         -9.65925826e-01,  8.66025404e-01, -5.00000000e-01],
        [ 1.47600000e+01,  1.74250000e+01,  6.20000000e+01, ...,
         -1.83697020e-16, -2.44929360e-16,  1.00000000e+00],
        [ 2.62400000e+01,  3.03050000e+01,  6.90000000e+01, ...,
          8.66025404e-01, -8.66025404e-01, -5.00000000e-01],
        ...,
        [ 1.47600000e+01,  1.74250000e+01,  5.00000000e+01, ...,
          7.07106781e-01, -2.44929360e-16,  1.00000000e+00],
        [ 2.29600000e+01,  2.65150000e+01,  6.40000000e+01, ...,
         -8.66025404e-01, -1.00000000e+00, -1.83697020e-16],
        [ 2.46000000e+01,  3.03050000e+01,  6.40000000e+01, ...,
         -1.00000000e+00,  5.00000000e-01, -8.66025404e-01]]),
 array([[ 1.72200000e+01,  2.12100000e+01,  5.00000000e+01, ...,
          5.00000000e-01, -5.00000000e-01,  8.66025404e-01],
        [ 1.80400000e+01,  2.19700000e+01,  6.20000000e+01, ...,
         -8.66025404e-01, -5.00000000e

In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [9]:
estimator = LogisticRegression()

cv = 4
cs = [0.01, 0.05, 0.1]
max_iters = [100, 200]

param_grid = {
    "C": cs,
    "max_iter": max_iters,
}

grid = GridSearchCV(estimator, param_grid, cv=cv, n_jobs=4, verbose=1)

grid.fit(X_train, y_train)

grid.best_estimator_

Fitting 4 folds for each of 6 candidates, totalling 24 fits


/Users/poslam/Downloads/projects/fefu/sem5/ds/.venv/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/Users/poslam/Downloads/projects/fefu/sem5/ds/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/poslam/Downloads/projects/fefu/sem5/ds/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED L

LogisticRegression(C=0.05, max_iter=200)

In [10]:
y_pred = grid.predict_proba(X_test)[:, -1]

In [13]:
r2 = r2_score(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)

print("r2: ", r2)
print("rmse: ", rmse)

r2:  -1.1125611170790322
rmse:  263.79891153943515


In [ ]:
X = pd.read_csv("bike_sharing_demand.csv")
y = X["count"]

X.drop(
    columns=["casual", "registered", "count"],
    inplace=True,
)

X.head(5)

***datetime*** - hourly date + timestamp  

***season*** -  1 = spring, 2 = summer, 3 = fall, 4 = winter

***holiday*** - whether the day is considered a holiday

***workingday*** - whether the day is neither a weekend nor holiday

***weather*** - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
    
***temp*** - temperature in Celsius

***atemp*** - "feels like" temperature in Celsius

***humidity*** - relative humidity

***windspeed*** - wind speed

***casual*** - number of non-registered user rentals initiated

***registered*** - number of registered user rentals initiated

***count*** - number of total rentals